# Методы сбора и обработки данных из сети Интернет
### Урок 2. Парсинг HTML. Библиотека Beautiful soup.
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

# https://hh.ru/search/vacancy?text=Data+scientist&area=1&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=50&no_magic=true&L_save_area=true&from=suggest_post
main_url = 'https://hh.ru'
vacancy = 'Data Scientist'
page = 0
all_vacancies = []
params = {'text': vacancy,
          'area': 1,
          'experience': 'doesNotMatter',
          'order_by': 'relevance',
          'search_period': 0,
          'items_on_page': 20,
          'page': page}
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)'
                         'Chrome/98.0.4758.141 YaBrowser/22.3.4.731 Yowser/2.5 Safari/537.36'}
response = requests.get(main_url + '/search/vacancy', params=params, headers=headers)
soup = bs(response.text, 'html.parser')
try:
    # Получаем последнюю страничку
    last_page = int(soup.find_all('a',{'data-qa':'pager-page'})[-1].text)
except:
    last_page = 1

for i in range(last_page):

    soup = bs(response.text, 'html.parser')

    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item'})

    for vacancy in vacancies:

        vacancy_info = {}
                vacancy_anchor = vacancy.find('a', {'data-qa': "vacancy-serp__vacancy-title"})
        vacancy_name = vacancy_anchor.getText()
        vacancy_info['name'] = vacancy_name

        vacancy_link = vacancy_anchor['href']
        vacancy_info['link'] = vacancy_link

        vacancy_info['site'] = main_url + '/'

        vacancy_salary = vacancy.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
        if vacancy_salary is None:
            min_salary = None
            max_salary = None
            currency = None
        else:
            vacancy_salary = vacancy_salary.getText()
            if vacancy_salary.startswith('РґРѕ'):
                max_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                min_salary = None
                currency = vacancy_salary.split()[-1]

            elif vacancy_salary.startswith('РѕС‚'):
                max_salary = None
                min_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

            else:
                max_salary = int("".join([s for s in vacancy_salary.split('вЂ“')[1] if s.isdigit()]))
                min_salary = int("".join([s for s in vacancy_salary.split('вЂ“')[0] if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

        vacancy_info['max_salary'] = max_salary
        vacancy_info['min_salary'] = min_salary
        vacancy_info['currency'] = currency

        all_vacancies.append(vacancy_info)

    params['page'] += 1
    response = requests.get(main_url + '/search/vacancy', params=params, headers=headers)
    print(len(all_vacancies))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


#### Момент с `user-Agent`
В хед хантер (HH) если не передать юзер агент, то он нам даст 404 ошибку. Обязательно нужно передовать. Второй момент - идёт редирект по региону в котором мы живём. Это не является фильтром поиска вакансии. Это нормально. Это разгрузка нагрузки по серверам HH. 

https://hh.ru/search/vacancy?text=python&area=1 -ссылка на поиск по вакансии python

Хед хантер ограничивает нас выдачью 2000 вакансий. Как бы мы к нему не обращались, хоть через апи, но больше 2000 вакансий хед хантер не выдаёт. Это особенность ресурса. Больше 2000 вакансий за один запрос не полуить!

Собрать информацию о том сколько страниц с вакансиями. Это исследовать элемент и найти число со страницами `<span>40</span>` . Или исследовать элемент кнопку "Дальше". `<a class="bloko-button" rel="nofollow" data-qa="pager-next" href="/search/vacancy?text=python&amp;area=1&amp;page=1&amp;hhtmFrom=vacancy_search_list"><span>дальше</span></a>`

##### Как взять URL для нужной вакансии
Смотрим элемент через инструменты разработчика
```
<a class="serp-item__title" data-qa="serp-item__title" target="_blank" href="https://hh.ru/analytics_source/vacancy/76390304?from=vacancy_search_list&amp;query=python&amp;requestId=1675239218829a88533135ef1c35229e&amp;totalVacancies=5151&amp;position=46&amp;source=vacancies">Специалист в отдел мониторинга и автоматизации (VBA и Python)</a>
```
Берём тэг `'a'` с атрибутом `'data-qa': 'vacancy-serp__vacancy-title'` и берём у этого тега значение атрибута `href`.

In [ ]:
d=vac.find_all('a', {'data-qa': 'vacancy-serp__vacancy-title'}, href=True)

##### Точка входа 
https://hh.ru/vacancy/75538170?from=vacancy_search_list&query=python
У нас есть ссылка со списком вакансий (20 шт на странице). В ссылке прописывается та вакансия, которую мы хотим в качестве параметра: `query=python`. 

https://hh.ru/search/vacancy?text=Python&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true
Ссылка на поиск по вакансии. Здесь также присутствует наша вакансия в виде параметра. `text=Python`

Много разных параметров в ссылке но их не желательно удалять. Браузер так не делает. Лучше максимально подделоваться под браузер. 

Один из вариантов как можно указать ссылку:

In [24]:
vacancy = 'python'
url = f'https://hh.ru/search/vacancy?text={vacancy}&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

Ньюанс связанный с количеством страниц. на странице 50 вакансий, но мы получаем не больше 20. Переделали страницу и сделали динамическую выдачу данных. Остальные 30 страниц выдачи получаются с помощью динамики. 

В настройках страницы ставим показывать 20 страниц и не теряем динамического контента. 

Посмотреть куда мы пришли

In [57]:
response.url 

'https://spb.hh.ru/search/vacancy?search_field=name&search_field=company_name&search_field=description&search_field=name&search_field=company_name&search_field=description&search_field=name&search_field=company_name&search_field=description&search_field=name&search_field=company_name&search_field=description&text=python&text=python&text=python&text=python&page=4&hhtmFrom=vacancy_search_list&search_field=name&search_field=company_name&search_field=description&text=python'




# Работа над парсером самостоятельная


In [9]:
! pip install html5lib

  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)


Загружаем библиотеки

In [18]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

In [19]:
headers = {'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
vacancy = 'python'
url = f'https://hh.ru/search/vacancy?text={vacancy}&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

В хед хантер (HH) если не передать юзер агент в заголовках, то он нам даст 404 ошибку. 

#### Get запрос 
Выполняем `get` запрос на сайт хедхантер по указанному выше `url` и указываем заголовки ` headers=headers`

In [ ]:
response = requests.get(url)
print(response)

In [35]:
response = requests.get(url, headers=headers)
print(response)

<Response [200]>


#### DOM
Получаем DOM . 

In [36]:
soup = BeautifulSoup(response.text, 'html.parser')

Ищем в инструментах разработчика необходимый нам элемент. Текст с профессией. 

`<a class="serp-item__title" data-qa="serp-item__title" target="_blank" href="https://hh.ru/analytics_source/vacancy/68077706?from=vacancy_search_list&amp;query=Python&amp;requestId=1675242703327964ca472d0a0c76effa&amp;totalVacancies=5168&amp;position=0&amp;source=vacancies">Python разработчик</a>`

Разбираем DOM на запчасти. Собираем тег `<a>` с классом `class="serp-item__title`. Внутри этого тега у нас есть ссылка на вакансию и название специальности. сама вакансия. 2 единицы информации необходимой нам хронится в каждом элементе списка. 

In [37]:
vacations = soup.find_all('a', {'class': 'serp-item__title'})

У нас собирается список со всеми тегами `<a>` Посмотрим длину списка, она должна быть равной 20, т.к на странице у нас показано 20 вакансий. 

In [38]:
len(vacations)

20

Посмотрим на то что мы собрали, Посмотрим на самый первый собранный элемент. 

In [39]:
vacations[0]

<a class="serp-item__title" data-qa="serp-item__title" href="https://hh.ru/vacancy/68077706?from=vacancy_search_list&amp;query=python" target="_blank">Python разработчик</a>

#### Собираем список со специальностями

In [40]:
vacancy_list = []
for el in vacations:
    vacancy_list.append(el.text)

In [ ]:
len(vacancy_list)

#### Собираем список со ссылками на специальности

In [42]:
vacancy_link = []
for el in vacations:
    vacancy_link.append(el['href'])

In [12]:
len(vacancy_link)

['https://hh.ru/vacancy/68077706?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/76362476?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/71389643?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/68918046?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/72161518?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/73984561?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/76418401?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/69072132?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/66075789?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/71324858?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/76486172?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/47892571?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=python',
 'https://hh.ru/vacancy/73595103?from=vacancy_searc

#### Создаю словарь, где ключ это название вакансии, значение - ссылка на вакансию. 

In [43]:
if len(vacancy_list) == len(vacancy_link):
    vacansion_and_href_dict = dict(zip(vacancy_list, vacancy_link))
else:
    print('Списки не равны по длине')

In [45]:
pprint(vacansion_and_href_dict)

{'Backend python (стажер)': 'https://hh.ru/vacancy/76223564?from=vacancy_search_list&query=python',
 'Build system Python developer': 'https://hh.ru/vacancy/76486868?from=vacancy_search_list&query=python',
 'Data science': 'https://hh.ru/vacancy/75914060?from=vacancy_search_list&query=python',
 'Junior Back-end Developer (Python)': 'https://hh.ru/vacancy/75702430?from=vacancy_search_list&query=python',
 'Junior разработчик нейросетевых алгоритмов / Reinforcement Learning': 'https://hh.ru/vacancy/75081176?from=vacancy_search_list&query=python',
 'Middle Python-разработчик': 'https://hh.ru/vacancy/75679088?from=vacancy_search_list&query=python',
 'Python Devops (DSCore)': 'https://hh.ru/vacancy/71389643?from=vacancy_search_list&query=python',
 'Python developer': 'https://hh.ru/vacancy/72424557?from=vacancy_search_list&query=python',
 'Python разработчик': 'https://hh.ru/vacancy/75791573?from=vacancy_search_list&query=python',
 'Python разработчик (удалённо)': 'https://hh.ru/vacancy/7595

In [51]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

headers = {'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
vacancy = input('Кем хотите работать?')
url = f'https://hh.ru/search/vacancy?text={vacancy}&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
vacations = soup.find_all('a', {'class': 'serp-item__title'})
print(f'Найдено {len(vacations)} вакансий')

vacs_link = []
vacs_list = []
for el in vacations:
    vacs_list.append(el.text)
    vacs_link.append(el['href'])

if len(vacs_list) == len(vacs_link):
    vacansion_and_href_dict = dict(zip(vacs_list, vacs_link))
else:
    print('Списки не равны по длине')
    
pprint(vacansion_and_href_dict)


Кем хотите работать?лор
Найдено 20 вакансий
{'Администратор клиники': 'https://hh.ru/vacancy/76514694?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Администратор медицинского центра': 'https://hh.ru/vacancy/75679094?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач - оториноларинголог детский': 'https://hh.ru/vacancy/74386211?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач ЛОР': 'https://hh.ru/vacancy/76073952?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач онколог отделения опухолей головы и шеи': 'https://hh.ru/vacancy/75632689?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач оториноларинголог (м. Щукинская)': 'https://hh.ru/vacancy/76480334?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач-оториноларинголог': 'https://hh.ru/vacancy/76245567?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач-оториноларинголог (ЛОР)': 'https://hh.ru/vacancy/76160993?from=vacancy_search_list&query=%D0%BB%D0%BE%D1%80',
 'Врач-оториноларин

### Пример решения домашнего задания
https://github.com/Androkotey

Простое и хорошее решение строковоми методами. 

In [56]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

# Функция для зарплаты
def salary_extraction(vacancy_salary):
    salary_dict = {'min': None, 'max': None, 'cur': None}

    if vacancy_salary:
        # делится всё по этому символу `' – '`?
        # удаляются пробелы и получаем общую структуру
        raw_salary = vacancy_salary.getText().replace(' – ', ' ').replace(' ', '').split()
        # Если в списке есть ДО
        if raw_salary[0] == 'до':
            # до 380 000 руб.
            # тогда сэлери макс берётся из первого элемента списка
            salary_dict['max'] = int(raw_salary[1])
        elif raw_salary[0] == 'от':
            # от 50 000 руб.
            # берём минимальную зарплату
            salary_dict['min'] = int(raw_salary[1])
        else:
            # 50 000 – 100 000 руб.
            salary_dict['min'] = int(raw_salary[0])
            salary_dict['max'] = int(raw_salary[1])
        salary_dict['cur'] = raw_salary[2].replace('.', '')

    return salary_dict


main_url = 'https://spb.hh.ru/'

params = {'search_field': ['name', 'company_name', 'description']}
params['text'] = input('Введите вакансию для поиска: ')
max_page = 99999  # не вижу смысла в ограничении количества страниц
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
page_link = '/search/vacancy'
vacancies = []
i = 0

# Снаружи внешний цикл
# который также завязан на следующей странице
# if not next_page

while True:
    response = requests.get(main_url + page_link,
                            params=params,
                            headers=headers)
    html = response.text
    soup = bs(html, 'html.parser')
    # собираются вакансии
    vacancies_soup = soup.find_all('div', {'class': ['vacancy-serp-item-body__main-info']})

    print(f'Page {i} is being processed...')
    for vacancy in vacancies_soup:

        vacancy_data = {'website': 'hh.ru'}

        vacancy_title = vacancy.find('a')

        vacancy_name = vacancy_title.getText()
        vacancy_link = vacancy_title['href'][: vacancy_title['href'].index('?')]
        vacancy_salary = salary_extraction(vacancy.find('span', {'class': ['bloko-header-section-3']}))
        vacancy_employer = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText().replace('\xa0', ' ')
        vacancy_address = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText().replace('\xa0', ' ')

        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = vacancy_link
        vacancy_data['salary_min'] = vacancy_salary['min']
        vacancy_data['salary_max'] = vacancy_salary['max']
        vacancy_data['salary_currency'] = vacancy_salary['cur']
        vacancy_data['employer'] = vacancy_employer
        vacancy_data['address'] = vacancy_address

        vacancies.append(vacancy_data)
# проверяем, если мы не нашли кнопку следующая или достигли максимальной страницы
# по параметру max_page можно ограничить число страниц для выдачи. 
    next_page = soup.find('a', {'data-qa': 'pager-next'})
    if not next_page or (i == max_page):
        break
    page_link = next_page['href']
    print(f'Page {i} done')
    i += 1

vacancies_data = pd.DataFrame(data=vacancies)
prefix = '_'.join(params['text'].split())
vacancies_data.to_csv(f'hh_vacancies_{prefix}.csv', index=False)


Введите вакансию для поиска: python
Page 0 is being processed...
Page 0 done
Page 1 is being processed...
Page 1 done
Page 2 is being processed...
Page 2 done
Page 3 is being processed...
Page 3 done
Page 4 is being processed...


AttributeError: 'NoneType' object has no attribute 'getText'

##  Пример решения домашнего задания
https://github.com/belkanov

In [55]:
"""
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность)
с сайтов HH(обязательно) и/или Superjob(по желанию).
Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы).
Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).
Структура должна быть одинаковая для вакансий с обоих сайтов.
Общий результат можно вывести с помощью dataFrame через pandas.
Сохраните в json либо csv.
"""
import re
from time import sleep

import requests
from bs4 import BeautifulSoup as bs
from collections import namedtuple
import logging
import json

RE_SALARY = re.compile(r'(?:\d+\s*){1,}')
RE_CURRENCY = re.compile(r'\D+')
MAIN_URL = 'https://hh.ru'
VACANCY_URL = f'{MAIN_URL}/search/vacancy'
HEADERS = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
}
vacancy_name = 'python'
PARAMS = {
    'text': vacancy_name
}

Salary = namedtuple('Salary', (
    'min',
    'max',
    'currency'
))

logging.basicConfig(format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger('job_scraper')
logger.setLevel(logging.INFO)

# если включить - можно увидеть редиректы
# requests_log = logging.getLogger("urllib3")
# requests_log.setLevel(logging.DEBUG)
# requests_log.propagate = True


def get_response(url, headers, params=None):
    timeouts = (5, 5)  # conn, read
    response = requests.get(url,
                            headers=headers,
                            params=params,
                            timeout=timeouts)
    if response.ok:
        # на случай редиректа hh.ru -> rostov.hh.ru
        splitted_response_url = response.url.split('/')
        new_main_url = f'{splitted_response_url[0]}//{splitted_response_url[2]}'
        return response, new_main_url

# Отдельная функция, которая преобразует полученные значения зарплаты к числовому типу данных
def get_int(re_match):
    return int(re_match.group().replace(' ', ''))

# Проверка зарплатных полей на None значения и обработка зарплат
def get_salary(tag):
    if tag is None:
        return Salary(None, None, None)

    text = clear_tag_text(tag)
    # получалось неплохо через всякие сплиты/слайсы/джоины,
    # но потом пришли 'бел. руб.' и все сломалось =)
    # поэтому регулярки
    if 'от' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(salary, None, re_currency.group())
    elif 'до' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(None, salary, re_currency.group())
    else:
        re_min_salary = RE_SALARY.search(text)
        min_salary = get_int(re_min_salary)
        re_max_salary = RE_SALARY.search(text, re_min_salary.end())
        max_salary = get_int(re_max_salary)
        re_currency = RE_CURRENCY.search(text, re_max_salary.end())
        return Salary(min_salary, max_salary, re_currency.group())
    
    
# Функция удаляет символы переноса строчек 

def clear_tag_text(tag):
    text = tag.getText()
    text = text.replace('\n', '')
    # внезапно вылезло много пробелов
    splitted = [word for word in text.split() if word]
    text = ' '.join(splitted)
    return text


def parse_vacancy_link(tag):
    link = tag.get('href')
    return f'{MAIN_URL}{link}'


def save_to_file(data, file_name):
    with open(file_name, 'w', encoding='utf8') as f:
        f.write(data)

# Сам алгоритм перебора данных. 
# делаем запрос по ссылке
# получаем вакансии

# Функция с циклом которая перебирает вакансии на одной странице.

def parse_response(response, main_url):
    vacancies_info = []

    soup = bs(response.text, 'html.parser')
    anchor = soup.find('div', {'class': 'vacancy-serp-content'})

    vacancy_results = anchor.find('div', {'data-qa': 'vacancy-serp__results'})
    # здесь получаются наши вакансии через find_all()
    vacancies = vacancy_results.find_all('div', {'class': 'vacancy-serp-item'})
    logger.info('Нашел %d вакансий. Обрабатываю...', len(vacancies))
    # начинаем итеррироваться по всему списку, 
    # заходя внутрь и получая данные по каждой вакансии
    for vacancy in vacancies:
        title_tag = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        salary_tag = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        
        # словарик со всей необходимой нам информацией. 
        # для каждого данного возвращается функция и всё нам "причёсывает"
        vacancy_info = {
            'name': clear_tag_text(title_tag),
            'salary': get_salary(salary_tag),
            'link': title_tag.get('href'),
            'site': main_url,
        }
        vacancies_info.append(vacancy_info)

    return vacancies_info, anchor


def main():
    all_vacancies = []

    page_cnt = 1
    url = VACANCY_URL
    headers = HEADERS
    params = PARAMS
    
    # Второй главный цикл, который перебирает страницы, 
    # обрабатывая каждую страницу с помощью функций
    while True:
        logger.info('Parse page #%d', page_cnt)
        response, main_url = get_response(url, headers=headers, params=params)
        # если ответ не пришёл
        if not response:
            logger.error('NO response from %s', url)
            raise SystemExit(1)
        
        try:
            vacancies_info, anchor = parse_response(response, main_url)
        except ValueError as e:
            logger.exception(e)
            save_to_file(response.text, 'error_response.html')
            raise SystemExit(1)

        all_vacancies.extend(vacancies_info)

        logger.info('Ищу следующую страницу...')
        next_link = anchor.find('a', {'data-qa': 'pager-next'})
        # переход по страницам как осуществляется? 
        # Ищем кнопку следующая на странице и ссылку которая под этой кнопкой бывает
        # извлекаем ссылку на следующую страницу
        
        # дальше по этой ссылке снова выполняем get-запрос.w
        if next_link:
            logger.info('Нашел.')
            url = f'{main_url}{next_link.get("href")}'
            params = None
            page_cnt += 1
            sleep(1)  # не будем спамить запросами
        else:
            logger.info('Видимо это последняя =) Всего обработано %d страниц',
                        page_cnt)
            break

    with open('vacancies.json', 'w', encoding='utf-8') as f:
        json.dump(all_vacancies, f, ensure_ascii=False)


if __name__ == '__main__':
    logger.info('--- START')
    main()
    logger.info('--- END')


2023-02-01 13:18:28 | INFO     | job_scraper | --- START
2023-02-01 13:18:28 | INFO     | job_scraper | Parse page #1
2023-02-01 13:18:30 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:18:30 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:18:30 | INFO     | job_scraper | Нашел.
2023-02-01 13:18:31 | INFO     | job_scraper | Parse page #2
2023-02-01 13:18:32 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:18:32 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:18:32 | INFO     | job_scraper | Нашел.
2023-02-01 13:18:33 | INFO     | job_scraper | Parse page #3
2023-02-01 13:18:35 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:18:35 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:18:35 | INFO     | job_scraper | Нашел.
2023-02-01 13:18:36 | INFO     | job_scraper | Parse page #4
2023-02-01 13:18:38 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю..

2023-02-01 13:20:19 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:20:19 | INFO     | job_scraper | Нашел.
2023-02-01 13:20:20 | INFO     | job_scraper | Parse page #32
2023-02-01 13:20:22 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:20:22 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:20:22 | INFO     | job_scraper | Нашел.
2023-02-01 13:20:23 | INFO     | job_scraper | Parse page #33
2023-02-01 13:20:25 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:20:25 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:20:25 | INFO     | job_scraper | Нашел.
2023-02-01 13:20:26 | INFO     | job_scraper | Parse page #34
2023-02-01 13:20:28 | INFO     | job_scraper | Нашел 0 вакансий. Обрабатываю...
2023-02-01 13:20:28 | INFO     | job_scraper | Ищу следующую страницу...
2023-02-01 13:20:28 | INFO     | job_scraper | Нашел.
2023-02-01 13:20:29 | INFO     | job_scraper | Parse page #35
2023

## Пример парсера, но изменилась структура сайта и парсер уже не отрабатывает 

In [13]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

headers = {'user-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B137 Safari/601.1'}
url = 'https://msk.hh.ru/search/vacancy'

def find_vacations(vac_name):
    ### Понимаю, что код с двумя вложенными циклами трудно назвать оптимальным, 
    ### но на что-то лучшее, к сожалению, не хватило времени. Очень большие выдачи считает
    ### довольно долго (5-7 мин)
    ### Максимальное число ответов в выдаче, независимо от их реального числа, - не более 2000. Видимо, опять сайт режет?
    ### Добавил в выдачу еще и работодателя, хотя этого не было в задании.
    params = {'text' : vac_name}
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    c = soup.find('h1', {'data-qa': 'bloko-header-3'})
    x = re.findall('[0-9]+', c.get_text())
    number = int(''.join(map(str, x)))
    number = int(number/20) #Нашел на странице строчку, где указано общее число запросов в выдаче, вынул оттуда это число, разделил на 20 (это значение числа вакансий #на странице установил принудительно ниже) и так нашел общее число страниц в выдаче
    
    lst = []
    page  = 0
    while page <= number:
        params = {'text' : vac_name, 'page' : page, 'items_on_page' : 20}
        response = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        vacations = soup.find_all('div', {'class': 'vacancy-serp-item'})
        for vac in vacations:
            temp = {}
            a=vac.find_all('span', {'class': 'g-user-content'})
            for elt in a:
                temp['Должность'] = elt.text
            b=vac.find_all('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            for elt in b:
                temp['Зарплата'] = elt.text.replace('\\u202f','')
            c=vac.find_all('a', {'data-qa': 'vacancy-serp__vacancy-employer'}) 
            for elt in c:
                temp['Работодатель'] = elt.text
            d=vac.find_all('a', {'data-qa': 'vacancy-serp__vacancy-title'}, href=True) 
            for elt in d:
                temp['Ссылка на вакансию'] = elt['href'] 
            lst.append(temp)
            page += 1
    x = pd.DataFrame(lst)

    x['Валюта'] = x['Зарплата'].replace('[0-9–дот]','',regex=True)
    x['Мин.зарплата'] = np.select([x['Зарплата'].str.contains('от',na=False), x['Зарплата'].str.contains('до',na=False), x['Зарплата'].str.contains('–',na=False)],
    [x['Зарплата'].replace('[^\\d]','',regex=True),0, x['Зарплата'].replace('–.*','',regex=True)])

    x['Макс.зарплата'] = np.select([x['Зарплата'].str.contains('от',na=False), x['Зарплата'].str.contains('до',na=False), x['Зарплата'].str.contains('–',na=False)],
    [0, x['Зарплата'].replace('[^\\d]','',regex=True), x['Зарплата'].replace('.*–','',regex=True).replace('[^\\d]','',regex=True)])
    x.fillna('не указана', inplace=True)
    x.drop('Зарплата',axis=1,inplace=True)
    x = x.reindex(columns=['Должность', 'Работодатель', 'Мин.зарплата', 'Макс.зарплата', 'Валюта', 'Ссылка на вакансию'])
    x[['Мин.зарплата', 'Макс.зарплата']] = x[['Мин.зарплата', 'Макс.зарплата']].astype('int')
    x.index +=1



# Собственное решение. Работа над решением


In [111]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

headers = {'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
vacancy = input('Кем хотите работать?')
url = f'https://hh.ru/search/vacancy?text={vacancy}&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

Кем хотите работать?механик по лифтам


Сделал DOM и поместил в переменную soup. Дальше с помощью инструментов разработчика смотрю на сайте контейнер, в котором находится всё что мне необходимо. Выделяю блок со всеми вакансиями со страницы.
`<div class="vacancy-serp-content">` - это тег который содержит вложенные теги с каждой вакансией. Предположим это будет стартовая точка, откуда я пойду спускаться вниз по детям. 

In [112]:
anchor = soup.find('div', {'class': 'vacancy-serp-content'})

Теперь работая с анкером в который поместил контейнер со всеми объявлениями, попробую найти контейнер под каждое объявление. В инструментах разработчика начинаю смотреть детей тега `<div class="vacancy-serp-content">`.  Я хочу найти все контейнеры, которые будут содержать информацию по каждой вакансии на странице. `<div class="serp-item" data-qa="vacancy-serp__vacancy vacancy-serp__vacancy_premium">` - это контейнер содержащий вакансию и её описание. Ищу все такие теги по классу `<div class="serp-item"`

In [113]:
vacansions_all_info = anchor.find_all('div', {'class': 'serp-item'})

Т.к. искал все вхождения `<div class="serp-item"` в результате должен получиться список.
Проверяю длину получившегося списка. Если длина равна 20 - это отличный результат, так как мы собираем по 20 вакансий со страницы. И теперь с каждым элементом этого списка можно работать, собрав нужную нам информацию. Циклом пройтись по этому списку например.

In [82]:
len(vacansions_all_info) 

100

Снова захожу в инструменты разработчика и начинаю анализировать содержимое контейнера с вакансией `<div class="serp-item"`
Ищем детей этого контейнера. Для начала хочу обработать одно объявление. 

In [83]:
vacansion_info = vacansions_all_info[0]

При анализе кода в инструменте разработчика нахожу участок кода, содержащий нужную информацию `<div class="vacancy-serp-item-body">` Посмотрим что там есть...  Есть тэг с ссылкой на вакансию и названием вакансии `<a class="serp-item__title" data-qa="serp-item__title" target="_blank" href="https://hh.ru/vacancy/68077706?from=vacancy_search_list&amp;query=python">Python разработчик</a>`. Также в этом контейнере есть тэг с роботодателем: `<a data-qa="vacancy-serp__vacancy-employer" class="bloko-link bloko-link_kind-tertiary" href="/employer/2136954?hhtmFrom=vacancy_search_list">Домклик</a>`. собираем соответственно эту информацию.

Получаю название вакансии, которое содержится в теге `<a class="serp-item__title" data-qa="serp-item__title" target="_blank" href="https://hh.ru/vacancy/68077706?from=vacancy_search_list&amp;query=python">Python разработчик</a>`

In [84]:
vacansion_name = vacansion_info.find('a',{ 'class':'serp-item__title'}).get_text()

In [85]:
vacansion_name

'Сантехник / Монтажник систем отопления'

Получаю ссылку на вакансию, которая также содержится в теге в атрибуте href  `<a class="serp-item__title" data-qa="serp-item__title" target="_blank" href="https://hh.ru/vacancy/68077706?from=vacancy_search_list&amp;query=python">Python разработчик</a>`

In [86]:
vacansion_link = vacansion_info.find('a',{ 'class':'serp-item__title'})['href']

In [87]:
vacansion_link

'https://hh.ru/vacancy/76568749?query=%D1%81%D0%B0%D0%BD%D1%82%D0%B5%D1%85%D0%BD%D0%B8%D0%BA'

Получаю название организаци. которая разместила объявление о вакансии. Эти данные находим в теге `<a data-qa="vacancy-serp__vacancy-employer" class="bloko-link bloko-link_kind-tertiary" href="/employer/2136954?hhtmFrom=vacancy_search_list">Домклик</a>`, откуда надо взять текст. 

In [88]:
vacancion_organization = vacansion_info.find('a',{ 'class':'bloko-link'}).get_text()

In [ ]:
vacancion_organization

Зарплата указана не на всех вакансиях, Но там где она указана попробуем её собрать. В инструменте разработчика смотрим где расположены данные по зарплатам `<span data-qa="vacancy-serp__vacancy-compensation" class="bloko-header-section-3">100 000 – 250 000 <!-- -->руб.</span>`

In [96]:
try:
    vacancion_money =  vacansion_info.find('span',{ 'data-qa':'vacancy-serp__vacancy-compensation'}).text
except:
     vacancion_money = 'зарплата не указана'

In [97]:
vacancion_money

'50\u202f000 – 50\u202f000 руб.'

а теперь в цикле собрать данных и вывести пользователю. промежуточный результат

In [114]:
for vacansion_info in vacansions_all_info:
    vacansion_name = vacansion_info.find('a',{ 'class':'serp-item__title'}).get_text()
    vacansion_link = vacansion_info.find('a',{ 'class':'serp-item__title'})['href']
    vacancion_organization = vacansion_info.find('a',{ 'class':'bloko-link'}).get_text()
    try:
        vacancion_money =  vacansion_info.find('span',{ 'data-qa':'vacancy-serp__vacancy-compensation'}).text
    except:
        vacancion_money = 'зарплата не указана'
    print(f' вакансия: {vacansion_name}\n организация: {vacancion_organization}\n ссылка на вакансию: {vacansion_link}\n зарплата: {vacancion_money} \n *******')

 вакансия: Наладчик по лифтам
 организация: ООО Мегаполис
 ссылка на вакансию: https://hh.ru/vacancy/76583048?from=vacancy_search_list&query=%D0%BC%D0%B5%D1%85%D0%B0%D0%BD%D0%B8%D0%BA+%D0%BF%D0%BE+%D0%BB%D0%B8%D1%84%D1%82%D0%B0%D0%BC
 зарплата: от 80 000 руб. 
 *******
 вакансия: Механик по лифтам
 организация: ООО Технолифт Сервис
 ссылка на вакансию: https://hh.ru/vacancy/75892171?from=vacancy_search_list&query=%D0%BC%D0%B5%D1%85%D0%B0%D0%BD%D0%B8%D0%BA+%D0%BF%D0%BE+%D0%BB%D0%B8%D1%84%D1%82%D0%B0%D0%BC
 зарплата: от 130 000 руб. 
 *******
 вакансия: Электромеханик по лифтам
 организация: ООО ТСК АРКС
 ссылка на вакансию: https://hh.ru/vacancy/76396321?from=vacancy_search_list&query=%D0%BC%D0%B5%D1%85%D0%B0%D0%BD%D0%B8%D0%BA+%D0%BF%D0%BE+%D0%BB%D0%B8%D1%84%D1%82%D0%B0%D0%BC
 зарплата: 60 000 – 60 000 руб. 
 *******
 вакансия: Электромеханик по лифтам
 организация: ООО Подъем
 ссылка на вакансию: https://hh.ru/vacancy/75675907?from=vacancy_search_list&query=%D0%BC%D0%B5%D1%85%D0%B0%D0%

# ВЫВОД В ОДНОМ СКРИПТЕ

In [123]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

headers = {'user-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
vacancy = input('Кем хотите работать?')
url = f'https://hh.ru/search/vacancy?text={vacancy}&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

anchor = soup.find('div', {'class': 'vacancy-serp-content'})
vacansions_all_info = anchor.find_all('div', {'class': 'serp-item'})
print(f'Показываю {len(vacansions_all_info)} вакансий \n')

for vacansion_info in vacansions_all_info:
    vacansion_name = vacansion_info.find('a',{ 'class':'serp-item__title'}).get_text()
    vacansion_link = vacansion_info.find('a',{ 'class':'serp-item__title'})['href']
    vacancion_organization = vacansion_info.find('a',{ 'class':'bloko-link'}).get_text()
    try:
        vacancion_money =  vacansion_info.find('span',{ 'data-qa':'vacancy-serp__vacancy-compensation'}).text
    except:
        vacancion_money = 'зарплата не указана'
        
    print(f'вакансия: {vacansion_name}\n'
          f'организация: {vacancion_organization}\n'
          f'ссылка на вакансию: {vacansion_link}\n'
          f'зарплата: {vacancion_money} \n *******')

Кем хотите работать?IT безопасность
Показываю 20 вакансий 

вакансия: Специалист по информационной безопасности
организация: ООО HeadHunter:: IT
ссылка на вакансию: https://hh.ru/vacancy/68891020?from=vacancy_search_list&query=IT+%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C
зарплата: зарплата не указана 
 *******
вакансия: Администратор систем информационной безопасности
организация: ООО HeadHunter:: IT
ссылка на вакансию: https://hh.ru/vacancy/71324797?from=vacancy_search_list&query=IT+%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C
зарплата: зарплата не указана 
 *******
вакансия: Инженер по информационной безопасности
организация: ООО HeadHunter:: IT
ссылка на вакансию: https://hh.ru/vacancy/68898061?from=vacancy_search_list&query=IT+%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C
зарплата: зарплата не указана 
 *******
вакансия: Начальник отдела IT-инфраструктуры
организация: НИИ Восход
ссылка на вакансию